##  감성체인 생성

In [1]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed

In [16]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名中国社会科学家，你的任务是分析关于“美日韩”三边关系的中国新闻报道，并从中国的视角进行情感分析。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。所有提供的句子都必须进行情感分析，不得遗漏任何句子。**
    **新闻文章的句子列表:**
    {sentences}

    **请仅返回与提供句子数量相匹配的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)


In [17]:
# 체인 구성
sentiment_chain = prompt_template | llm

## 데이터 로드

In [4]:
import json

# JSON 파일 로드
file_path = "../data/smaple.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 13개의 기사 로드 완료


## 감성분석 수행 함수

In [12]:
def analyze_sentiment(article):
    """기사 전체를 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        response = sentiment_chain.invoke({"sentences": "\n".join(article["sentences"])})
        return response.content.strip()
    except Exception as e:
        return {"error": str(e)}  # 오류 발생 시 오류 메시지 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    article["sentiment_responses"] = sentiment_response

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            _, score = line.split(":")
            sentiment_scores.append(float(score.strip()))
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article


def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    #기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            article["sentiment_responses"] = future.result()  # 감성 분석 결과 추가
            processed_article = process_article(article)  # 후처리
            results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [18]:
output_path = "../data/sentiment_analysis_results4.json"
run_sentiment_analysis(data, output_path)

 '美日韩举行首次联合军演乔治·华盛顿号航母参加' 분석 완료.
 '前原诚司在韩国发表演讲称美日韩并非要“封杀”中国 - 国际新闻' 분석 완료.
 '美日韩回应朝鲜提议美称要看行动' 분석 완료.
 '美日韩军机在日本海附近进行夜间飞行演练-新华网' 분석 완료.
 '日美韩外长拟下周开会以推动缓和日韩紧张关系' 분석 완료.
 '美日韩将举行三国首脑会谈或不涉及历史问题' 분석 완료.
 '美日韩正式签朝核与导弹情报交流协议即刻生效' 분석 완료.
 '日防相正式宣布将时隔4年与韩防长举行会谈 - 军事- 人民网' 분석 완료.
 '美情报高官:安倍应把竹岛主权交给韩国' 분석 완료.
 '邢海明大使接受韩国《每日经济新闻》专访-1' 분석 완료.
 '专家称美日韩军演为向中国传达一些信息' 분석 완료.
 '邢海明大使接受韩国《每日经济新闻》专访-2' 분석 완료.
 '邢海明大使接受韩国《每日经济新闻》专访-3' 분석 완료.
 전체 기사 감성 분석 완료. ../data/sentiment_analysis_results4.json
